<font size=6><b>Lec00. ARIMA  : Multivariate Time Series (MTS)</b></font><br>
* ref : https://www.analyticsvidhya.com/blog/2018/09/multivariate-time-series-guide-forecasting-modeling-python-codes/

# AirQualityUCI
* ref : https://archive.ics.uci.edu/ml/datasets/Air+Quality
* ref : https://gist.github.com/shreyasiitr/57f8fa30fa20b049359fb567cc6407d0
<pre>
0 Date (DD/MM/YYYY)
1 Time (HH.MM.SS)
2 True hourly averaged concentration CO in mg/m^3 (reference analyzer)
3 PT08.S1 (tin oxide) hourly averaged sensor response (nominally CO targeted)
4 True hourly averaged overall Non Metanic HydroCarbons concentration in microg/m^3 (reference analyzer)
5 True hourly averaged Benzene concentration in microg/m^3 (reference analyzer)
6 PT08.S2 (titania) hourly averaged sensor response (nominally NMHC targeted)
7 True hourly averaged NOx concentration in ppb (reference analyzer)
8 PT08.S3 (tungsten oxide) hourly averaged sensor response (nominally NOx targeted)
9 True hourly averaged NO2 concentration in microg/m^3 (reference analyzer)
10 PT08.S4 (tungsten oxide) hourly averaged sensor response (nominally NO2 targeted)
11 PT08.S5 (indium oxide) hourly averaged sensor response (nominally O3 targeted)
12 Temperature in Â°C
13 Relative Humidity (%)
14 AH Absolute Humidity

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings(action='ignore')

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set()

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)


# Data Load

In [39]:
df = pd.read_csv("./dataset/AirQualityUCI.csv", parse_dates=[['Date', 'Time']], sep=";")
print(df.shape)
print(df.info())
df.head()

(9471, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9471 entries, 0 to 9470
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date_Time      9471 non-null   object 
 1   CO(GT)         9357 non-null   object 
 2   PT08.S1(CO)    9357 non-null   float64
 3   NMHC(GT)       9357 non-null   float64
 4   C6H6(GT)       9357 non-null   object 
 5   PT08.S2(NMHC)  9357 non-null   float64
 6   NOx(GT)        9357 non-null   float64
 7   PT08.S3(NOx)   9357 non-null   float64
 8   NO2(GT)        9357 non-null   float64
 9   PT08.S4(NO2)   9357 non-null   float64
 10  PT08.S5(O3)    9357 non-null   float64
 11  T              9357 non-null   object 
 12  RH             9357 non-null   object 
 13  AH             9357 non-null   object 
 14  Unnamed: 15    0 non-null      float64
 15  Unnamed: 16    0 non-null      float64
dtypes: float64(10), object(6)
memory usage: 1.2+ MB
None


,Date_Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004 18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578",NaN,NaN
1,10/03/2004 19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255",NaN,NaN
2,10/03/2004 20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502",NaN,NaN
3,10/03/2004 21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867",NaN,NaN
4,10/03/2004 22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888",NaN,NaN


In [2]:
df.isna().sum()

Date_Time           0
CO(GT)            114
PT08.S1(CO)       114
NMHC(GT)          114
C6H6(GT)          114
PT08.S2(NMHC)     114
NOx(GT)           114
PT08.S3(NOx)      114
NO2(GT)           114
PT08.S4(NO2)      114
PT08.S5(O3)       114
T                 114
RH                114
AH                114
Unnamed: 15      9471
Unnamed: 16      9471
dtype: int64

In [3]:
df = df[:9357]
df.tail(2)

,Date_Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
9355,04/04/2005 13.00.00,"2,1",1003.0,-200.0,"9,5",961.0,235.0,702.0,156.0,1041.0,770.0,"28,3","13,5","0,5139",NaN,NaN
9356,04/04/2005 14.00.00,"2,2",1071.0,-200.0,"11,9",1047.0,265.0,654.0,168.0,1129.0,816.0,"28,5","13,1","0,5028",NaN,NaN


# 가공 & 전처리

## data , index 분리

In [4]:
df['Date_Time'] = pd.to_datetime(df.Date_Time , format = '%d/%m/%Y %H.%M.%S')

In [5]:
data = df.drop(['Date_Time'], axis=1)
data.index = df.Date_Time

## 타입변환, 불필요컬럼 삭제
<pre>
* Unnamed: 15	Unnamed: 16    --> drop
* CO(GT), C6H6(GT), T, RH, AH   --> 13,1    

In [6]:
data[['Unnamed: 15','Unnamed: 16']].value_counts()

Series([], dtype: int64)

In [7]:
data = data.drop(['Unnamed: 15','Unnamed: 16'], axis=1)

In [55]:
# data[['CO(GT)', 'C6H6(GT)', 'T', 'RH', 'AH']] = data[['CO(GT)', 'C6H6(GT)', 'T', 'RH', 'AH']].replace(',','.')
data = data.replace(regex=r',', value='.')
data.head(2)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
Date_Time,,,,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255


In [9]:
# data = data[['PT08.S1(CO)', 'NMHC(GT)', 'PT08.S2(NMHC)','NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)']].astype('float64')   
data = data.astype('float64')  

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9357 entries, 2004-03-10 18:00:00 to 2005-04-04 14:00:00
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CO(GT)         9357 non-null   float64
 1   PT08.S1(CO)    9357 non-null   float64
 2   NMHC(GT)       9357 non-null   float64
 3   C6H6(GT)       9357 non-null   float64
 4   PT08.S2(NMHC)  9357 non-null   float64
 5   NOx(GT)        9357 non-null   float64
 6   PT08.S3(NOx)   9357 non-null   float64
 7   NO2(GT)        9357 non-null   float64
 8   PT08.S4(NO2)   9357 non-null   float64
 9   PT08.S5(O3)    9357 non-null   float64
 10  T              9357 non-null   float64
 11  RH             9357 non-null   float64
 12  AH             9357 non-null   float64
dtypes: float64(13)
memory usage: 1023.4 KB


## missing value

In [11]:
cols = data.columns
print(cols)

for col in cols:
    for i in range(0,len(data)):
        if data[col][i] == -200:
            #print(col, i,  data[col][i])
            data[col][i] = data[col][i-1].copy()
            

Index(['CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)',
       'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)',
       'T', 'RH', 'AH'],
      dtype='object')


# checking stationarity
* ref : https://blog.naver.com/yonxman/220904870137
<pre>
* johansen : 경제변수들간의 공적분 검정
               AR 모형에 대한 가설검정을 통해 적분계열간 안정적인 장기 균형관계가 존재하는지를 점검하는 방법
* 단위근 검정 : 종속변수(Δyt)와 설명변수(yt-1) 간의 상관관계 존재유무를 나타내는 φ의 유의성 파악
* VAR 모형을 이용한 공적분 검정 : 두 벡터 ΔYt와 설명변수 Yt-1 간의 정규 상관계수(ρ)를 분석하여 통계량을 산출             

In [12]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped in the next iteration, 
#I would drop another and check the eigenvalues

johan_test_temp = data.drop([ 'CO(GT)'], axis=1)
coint_johansen(johan_test_temp,-1,1).eig

c:\ai\pythonproject\venv\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


array([1.75510896e-01, 1.52389933e-01, 1.15120416e-01, 1.04126281e-01,
       9.29485509e-02, 6.89397159e-02, 5.77070988e-02, 3.43554214e-02,
       3.05980659e-02, 1.18697142e-02, 2.46766099e-03, 7.09584856e-05])

# 학습

## train, test 분리

In [13]:
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

In [19]:
freq=train.index.inferred_freq
freq

'H'

In [33]:
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train,freq='H')
model_fit = model.fit()
pred = model_fit.forecast(model_fit.y, steps=len(valid))
pred[:1]

array([[8.87831181e-01, 8.41930040e+02, 2.71627237e+02, 1.98242516e+00,
        5.95810006e+02, 1.36652961e+02, 1.11978584e+03, 8.78809405e+01,
        8.29685784e+02, 5.45724682e+02, 1.05475985e+01, 3.49086438e+01,
        4.37327493e-01]])

In [48]:
pred_df = pd.DataFrame(data=pred, index=range(0,len(pred)),columns=[cols])
pred_df.head()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,0.887831,841.930040,271.627237,1.982425,595.810006,136.652961,1119.785840,87.880940,829.685784,545.724682,10.547598,34.908644,0.437327
1,0.991857,866.300135,269.296720,2.327046,619.651929,157.767278,1098.690737,90.574363,856.150818,598.061035,9.826373,37.449595,0.442717
2,1.104181,890.877509,267.701932,2.823187,645.301670,177.409556,1078.877607,93.129588,885.360153,648.593798,9.214091,39.678858,0.447482
3,1.219169,914.856579,266.614149,3.396825,671.288989,195.619763,1060.434050,95.567038,915.323932,696.956399,8.693913,41.640506,0.451830
4,1.332886,937.732765,265.865337,3.999277,696.669347,212.431288,1043.389748,97.891452,944.774167,742.857863,8.252727,43.370069,0.455894


## 평가

In [42]:
pred[0]

array([8.87831181e-01, 8.41930040e+02, 2.71627237e+02, 1.98242516e+00,
       5.95810006e+02, 1.36652961e+02, 1.11978584e+03, 8.78809405e+01,
       8.29685784e+02, 5.45724682e+02, 1.05475985e+01, 3.49086438e+01,
       4.37327493e-01])

In [46]:
valid.iloc[0].values

array([7.000e-01, 8.330e+02, 2.750e+02, 2.000e+00, 5.840e+02, 1.070e+02,
       1.144e+03, 8.000e+01, 8.210e+02, 4.630e+02, 1.130e+01, 3.250e+01,
       4.334e-01])

In [51]:
for c in cols:
    print(c)

CO(GT)
PT08.S1(CO)
NMHC(GT)
C6H6(GT)
PT08.S2(NMHC)
NOx(GT)
PT08.S3(NOx)
NO2(GT)
PT08.S4(NO2)
PT08.S5(O3)
T
RH
AH


In [50]:
np.sqrt(mean_squared_error(pred[0], valid.iloc[0].values))

25.845025458826267

In [53]:
#check rmse
for c in cols:
    print(f"RMSE: {c} \t {np.sqrt(mean_squared_error(pred_df[c], valid[c]))}") 

RMSE: CO(GT) 	 1.4086888836873896
RMSE: PT08.S1(CO) 	 205.89558284023104
RMSE: NMHC(GT) 	 6.67354871134665
RMSE: C6H6(GT) 	 7.130087248706202
RMSE: PT08.S2(NMHC) 	 277.84844376802596
RMSE: NOx(GT) 	 214.78322340911566
RMSE: PT08.S3(NOx) 	 244.9576966193666
RMSE: NO2(GT) 	 66.69695211709882
RMSE: PT08.S4(NO2) 	 490.0838893411126
RMSE: PT08.S5(O3) 	 446.5154164888281
RMSE: T 	 10.721325795600006
RMSE: RH 	 17.111676248173158
RMSE: AH 	 0.5216247245185668


In [54]:
#make final predictions
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

[[2.34596328e+00 1.08633212e+03 2.80762173e+02 1.24130779e+01
  1.05535947e+03 2.80882233e+02 6.59534851e+02 1.68444418e+02
  1.15918056e+03 8.50845529e+02 2.73639014e+01 1.55311062e+01
  5.15317053e-01]]


c:\ai\pythonproject\venv\lib\site-packages\statsmodels\tsa\base\tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
c:\ai\pythonproject\venv\lib\site-packages\statsmodels\base\wrapper.py:35: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


# 검증
* ref : https://www.statsmodels.org/dev/vector_ar.html